In [29]:
%matplotlib inline
# import all shogun classes
from shogun import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
import time
import os
import pandas as pd

In [30]:
import stylised_facts_data_utilities.createLOB as createLOB

In [31]:
etfDir = '/media/ak/My Passport1/MarketData/ETF_Levels/'
etfSymbols = sorted(os.listdir(etfDir))


In [32]:
etfSymbol_to_number = {label: i-1 for i, label in enumerate(set(etfSymbols), 1)}


In [33]:
uniqueETFList = list(etfSymbol_to_number.values()) #converting dictionary to ETF list

In [34]:
import random
twoETFS = random.sample(set(uniqueETFList), 2)

In [35]:

print('Pick the', twoETFS[0],' and ' ,twoETFS[1], 'symbols to play with')
### combine symbol index with etfDir to get all the location
for etfIdx in list(twoETFS):
    etfSymbolFiles = os.path.join(etfDir, etfSymbols[etfIdx])

    print('Symbol Location:', etfSymbolFiles)
    ## and make a list of all the dates
    listETFSymbolFiles = os.listdir(etfSymbolFiles)

    print('Picked the following ETF: ',etfSymbols[etfIdx])
    combinedETFfiles_A = dict()
    for fileIdx, _ in enumerate(os.listdir(etfSymbolFiles)):
        etfFileDate = os.listdir(etfSymbolFiles)[fileIdx]
        fileLoc = os.path.join(etfSymbolFiles, os.listdir(etfSymbolFiles)[fileIdx])
        combinedETFfiles_A[etfFileDate.split('.')[0]] = pd.read_csv(fileLoc)
        for idxKey, dateKey in enumerate(etfDateKeys):
            originalCreateLOB = createLOB.formatETFlob(combinedETFfiles_A[etfDateKeys[idxKey]])
            microPxDict[dateKey] = originalCreateLOB['MicroPriceReturns']
            durationsDict[dateKey] = originalCreateLOB['milliSeconds'].diff().fillna(0)

Pick the 8  and  7 symbols to play with
Symbol Location: /media/ak/My Passport1/MarketData/ETF_Levels/VWO
Picked the following ETF:  VWO
Symbol Location: /media/ak/My Passport1/MarketData/ETF_Levels/SPY
Picked the following ETF:  SPY


{'20120402':           millis     bid     ask  l1_bid_size  l1_ask_size   price  volume
 0       34200012  140.64  140.65           42           37  140.64     200
 1       34200012  140.64  140.65           42           37  140.64     600
 2       34200012  140.64  140.65           42           37  140.64     100
 3       34200014  140.64  140.65           42           37  140.64     100
 4       34200014  140.64  140.65           42           37  140.64     100
 ...          ...     ...     ...          ...          ...     ...     ...
 336812  57657510  141.77  141.78           20          330  141.78     153
 336813  57657510  141.77  141.78           20          330  141.78     100
 336814  57657592  141.77  141.78           20          330  141.78     953
 336815  57657592  141.77  141.78           20          330  141.78     247
 336816  57659200  141.77  141.78           25          330  141.77     300
 
 [336817 rows x 7 columns],
 '20120403':           millis     bid     ask 

In [ ]:
combinedETFfiles_A = dict() # combine all the files for a single ETF
# combinedETFfiles_B = dict() # combine all the files for a single ETF



    '''
    go through all the dates for the symbol choice above and create a location for each 
    etf symbol+ date csv and read it into a pandas dataframe then store it
    
    '''
    etfFileDate = os.listdir(etfSymbolFiles)[fileIdx]
    fileLoc = os.path.join(etfSymbolFiles, os.listdir(etfSymbolFiles)[fileIdx])
    combinedETFfiles_A[etfFileDate.split('.')[0]] = pd.read_csv(fileLoc)

In [ ]:
etfDateKeys =list(combinedETFfiles_A.keys())
print('This file has ',len(etfDateKeys), 'days')

In [ ]:
microPxDict = dict()
durationsDict = dict()

for idxKey, dateKey in enumerate(etfDateKeys):
    originalCreateLOB = createLOB.formatETFlob(combinedETFfiles_A[etfDateKeys[idxKey]])
    microPxDict[dateKey] = originalCreateLOB['MicroPriceReturns']
    durationsDict[dateKey] = originalCreateLOB['milliSeconds'].diff().fillna(0)

In [ ]:
dfConcat = pd.concat(list(microPxDict.values() ), axis =0, keys = list(microPxDict.keys()))

In [ ]:
microPriceArray = dfConcat.dropna().values
mPnonZero = microPriceArray[microPriceArray!=0]

In [ ]:
pd.DataFrame(mPnonZero).describe()

In [ ]:
subset =mPnonZero[0:15000]

In [ ]:
msk = np.random.rand(len(subset)) < 0.55
train = subset[msk]

test = subset[~msk]
print(train.shape, test.shape)

In [ ]:
features_p  =  RealFeatures(train.reshape(1,len(train)))
features_q = RealFeatures(test.reshape(1,len(test)))

In [ ]:
mmd = LinearTimeMMD()
kernel = GaussianKernel(10, 1)
mmd.set_kernel(kernel)
mmd.set_p(features_p)
mmd.set_q(features_q)
mmd.set_num_samples_p(1000)
mmd.set_num_samples_q(1000)
alpha = 0.05

In [ ]:
mmd.set_statistic_type(ST_BIASED_FULL)
statistic = mmd.compute_statistic()

In [ ]:
threshold = mmd.compute_threshold(alpha)
p_value = mmd.compute_p_value(statistic)
print(p_value)

In [ ]:
X = train
Y = test
plt.hist(X, alpha=0.5)
plt.hist(Y, alpha=0.5);

In [ ]:
import tqdm as tqdm

In [ ]:

def simple_statistic(X,Y, squared=False):
    assert X.ndim == Y.ndim == 1
    
    # IMPLEMENT: compute mean difference of X and Y
    result = np.mean(X) - np.mean(Y)
    
    if squared:
        result *= result
    return result

my_statistic = simple_statistic(X,Y)
print("Mean differencce:", my_statistic)


In [ ]:
def two_sample_permutation_test(test_statistic, X, Y, num_permutations, prog_bar=True):
    assert X.ndim == Y.ndim
    
    statistics = np.zeros(num_permutations)
    
    range_ = range(num_permutations)
    if prog_bar:
        range_ = tqdm(range_)
    for i in range_:
        # concatenate samples
        if X.ndim == 1:
            Z = np.hstack((X,Y))
        elif X.ndim == 2:
            Z = np.vstack((X,Y))
            
        # IMPLEMENT: permute samples and compute test statistic
        perm_inds = np.random.permutation(len(Z))
        Z = Z[perm_inds]
        X_ = Z[:len(X)]
        Y_ = Z[len(X):]
        my_test_statistic = test_statistic(X_, Y_)
        statistics[i] = my_test_statistic
    return statistics

num_permutations = 200
statistics = two_sample_permutation_test(simple_statistic, X, Y, num_permutations,prog_bar=False)

In [ ]:
def plot_permutation_samples(null_samples, statistic=None):
    plt.hist(null_samples)
    plt.axvline(x=np.percentile(null_samples, 2.5), c='b')
    legend = ["95% quantiles"]
    if statistic is not None:
        plt.axvline(x=statistic, c='r')
        legend += ["Actual test statistic"]
    plt.legend(legend)
    plt.axvline(x=np.percentile(null_samples, 97.5), c='b')
    plt.xlabel("Test statistic value")
    plt.ylabel("Counts")
    
plot_permutation_samples(statistics, my_statistic)

In [ ]:
from scipy.spatial.distance import squareform, pdist, cdist

def sq_distances(X,Y=None):
    """
    If Y=None, then this computes the distance between X and itself
    """
    assert(X.ndim==2)

    # IMPLEMENT: compute pairwise distance matrix. Don't use explicit loops, but the above scipy functions
    # if X=Y, use more efficient pdist call which exploits symmetry
    if Y is None:
        sq_dists = squareform(pdist(X, 'sqeuclidean'))
    else:
        assert(Y.ndim==2)
        assert(X.shape[1]==Y.shape[1])
        sq_dists = cdist(X, Y, 'sqeuclidean')

    return sq_dists

def gauss_kernel(X, Y=None, sigma=1.0):
    """
    Computes the standard Gaussian kernel k(x,y)=exp(- ||x-y||**2 / (2 * sigma**2))

    X - 2d array, samples on left hand side
    Y - 2d array, samples on right hand side, can be None in which case they are replaced by X
    
    returns: kernel matrix
    """

    # IMPLEMENT: compute squared distances and kernel matrix
    sq_dists = sq_distances(X,Y)
    K = np.exp(-sq_dists / (2 * sigma**2))
    return K

# IMPLEMENT
def linear_kernel(X, Y):
    return np.dot(X, Y.T)

In [ ]:
def quadratic_time_mmd(X,Y,kernel):
    assert X.ndim == Y.ndim == 2
    K_XX = kernel(X,X)
    K_XY = kernel(X,Y)
    K_YY = kernel(Y,Y)
       
    n = len(K_XX)
    m = len(K_YY)
    
    # IMPLEMENT: unbiased MMD statistic (could also use biased, doesn't matter if we use permutation tests)
    np.fill_diagonal(K_XX, 0)
    np.fill_diagonal(K_YY, 0)
    mmd = np.sum(K_XX) / (n*(n-1))  + np.sum(K_YY) / (m*(m-1))  - 2*np.sum(K_XY)/(n*m)
    return mmd

In [ ]:
my_kernel = lambda X,Y : gauss_kernel(X,Y,sigma=0.3)
my_mmd = lambda X,Y : quadratic_time_mmd(X[:,np.newaxis],Y[:,np.newaxis], my_kernel)

statistics = two_sample_permutation_test(my_mmd, X, Y, num_permutations, prog_bar=False)
my_statistic = my_mmd(X,Y)

plot_permutation_samples(statistics, my_statistic)